<a href="https://colab.research.google.com/github/Payalpat25/3601/blob/main/Another_copy_of_CGSC_3601_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Welcome to assignment 1. Here, we'll be building on our frog by adding a **declarative memory**, so it can solve a more complex problem.

Our frog's hunger is never quite sated. Desperate for more flies, it rushes to the cellar for more. But our frog doesn't remember where it left its key! The frog's keys are **colour-coded** to the doors they open, so the frog must:

1. Look at the door, and remember its colour,
2. Go to one of the keys, and check if it matches the door,
3. If it does not match, go to the other key,
4. Pick up the key matching the door, and
5. Go back to the door and open it

This assignment has two parts:

1. This code (20 points)
  - Evaluation is based on how often the frog manages to **open the door**. Every 5% success rate is worth one point.
  - **Thus, if the frog succeeds 75% of the time, you get all 20 points.**
  - If the frog succeeds 100% of the time, **you get 5 bonus points**
2.   Short answer questions (80 points)
  - Short answer questions will be available as a brightspace quiz
  - If you work in groups, you should work on them together. Only one group member needs to complete the quiz
  - The quiz will be available within a few days of the programming portion
  - There will be **5 short answer questions**, graded from **0 to 4** (half points are possible)
  - Your final score for the short answer questions will be 4 * the sum of your answer scores

### First, run this

This installs the `python_actr` library, which we use to create the frog's mind, and the `pondworld` environment from my github. `pondworld` provides the tools we need to build and display the world the frog lives in.

In [ ]:
%pip install python_actr git+https://github.com/eilene-ftf/pondworld.git


  Cloning https://github.com/eilene-ftf/pondworld.git to /tmp/pip-req-build-tzf6apm3
  Running command git clone --filter=blob:none --quiet https://github.com/eilene-ftf/pondworld.git /tmp/pip-req-build-tzf6apm3
  Resolved https://github.com/eilene-ftf/pondworld.git to commit 7aa712ff4d7ad0034d2e95090831542552845893
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00
  Created wheel for python_actr: filename=python_actr-1.9.2-py3-none-any.whl size=30911 sha256=849f8c7b1e176cc61

In [ ]:
%pip install python_actr git+https://github.com/eilene-ftf/pondworld.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eilene-ftf/pondworld.git to /tmp/pip-req-build-527ib3l9
  Running command git clone --filter=blob:none --quiet https://github.com/eilene-ftf/pondworld.git /tmp/pip-req-build-527ib3l9
  Resolved https://github.com/eilene-ftf/pondworld.git to commit 660c9ca8d24a36ce7cea3737103bb1ea81630156
  Preparing metadata (setup.py) ... done


### Imported Libraries

These are libraries that support our frog and its environment. The first two are `gymnasium`, a software library for creating simulated environments for AIs, and `minigrid`, a collection of environments using gymnasium.

After that is `pondworld`, which uses `minigrid` environments to make the frog's world.

`python_actr` is what you use to create the frog's mind, you use it to model how the frog behaves in its world. `python_actr` is a model all on its own, but it can also be a scaffold for other AI models.

The final two libraries, `IPython` and `time`, are used to control the text output. `time` allows us to make the frog wait, while `IPython` allows us to refresh the output.

In [ ]:
import gymnasium as gym
from minigrid.minigrid_env import MiniGridEnv

import pondworld
from pondworld import FrogControl, EnvState

from python_actr import Model, ACTR, Buffer, Memory, log_everything

from IPython.display import clear_output, Audio, display

import time

### The Frog's Environment

ACT-R models need ACT-R environments to live in, this frog lives in a pond. This very basic environment will have an interface for the frog to interact with its world placed in it.

This time, we will place the `frog_body` and the `env` (the frog's world) in the environment, when we define it (though it doesn't really matter whether we add them now, or later as we were doing previously).

This time we are using the `"cellar"` world, a tiny maze containing two keys and a locked door.

In [ ]:
class Pond(Model):
    # A function that allows the frog to refresh our view every time it looks around
    def clear(self):
        clear_output(wait=True)
    # Sets up the frog's virtual world, this virtual world has a locked cellar
    env = gym.make("cellar-v0")
    # Gives the frog a body in its world
    frog_body = FrogControl(env, textmode=True, emojis=True)

    # Initializes the frog's world
    start_world = frog_body.start


### The Ever-Important Frog

Here, you've got a minimal frog. It knows how to look around, open doors, and stop when it has successfully opened the door to its cellar. You are going to define the rest of its behaviour. You'll need to have the frog move around and pick up a key and bring it back to the door to unlock it.

Quite a lot has been added to this new frog. Instead of *one* working memory `Buffer`, it has *four*. Each one serves a specific purpose:

  - The `vision` buffer works as before, but the frog can now see **walls** and **the colour of objects**.
  - The `goal` buffer separates the frog's goals from what it's seeing. It stores whatever objective is the subject of the frog's attention currently.
  - The `recall` buffer is connected to `memory`, which stores facts that the frog knows about the world. When you use `memory.request()`, whatever you request will become available in `recall` on the next cycle.

There are also a few more possibilities. These new ones just tell the frog about the walls to its left, to its right, and ahead. It might be surrounded by walls or in a corner, and it needs to be able to understand that.

The frog is set up to always start by going `"left"`. What should the frog do first?

### How Memory Works

We can think of memory like a bag of `chunk`s, and we retrieve them by pattern matching, just like with the conditions of our productions.

We will need the following:

- `memory.add(item)` puts `item` into memory so that it can be retrieved later
- `memory.request(pattern)` retrieves an item that matches pattern, and places it in `recall`. Just like with production rules, if multiple chunks in memory match the pattern, one will be retrieved at random.

In [ ]:
# create our frog's brain
class FrogMind(ACTR):
    # give our frog a buffer that holds information
    # let's call this buffer "vision". it represents what the frog is seeing.
    vision=Buffer()
    goal=Buffer() # will store the frog's current goal
    recall=Buffer() # stores what the frog is remembering, such as door colour

    memory=Memory(recall)

    # here we're going to set what the frog is seeing right at the moment
    # initially we don't know what the frog is looking at
    vision.set('unknown')
    goal.set('key') # Sets the initial goal to go left and find a key
    recall.set('nothing') # we're not remembering anything at first

    # "possibilities" has been replaced with a set of slot names, corresponding
    # to the three items the frog might track in its world, and a set of values,
    # corresponding to what states those items might be in
    slots = ["fly", "key", "door", "wall"]
    values = ["nowhere",      # used when a wall or key or door in NOT in sight
              "here",         # used when a wall or key or door is directly in front
              "ahead",        # used when a key or door is visible ahead, never used for walls
              "right",        # used when a wall or key or door is to the left
              "left",         # used when a wall or key or door is to the right
              "held",         # only used when the frog is holding the key
              'right_corner', # used if there's a wall in front and to the left
              'left_corner',  # used if there's a wall in front and to the right
              'hall'] # only used if there's walls to the left AND right

    # also, we're keeping track of all the things the frog can do
    # you never know when you might need them
    actions = ["forward", "left", "right", "interact", "pickup"]


    ######
    # THE BASICS

    # look to see if there's any flies
    def look(vision='unknown', goal='?objective'):
        self.wait(0.6) # adds a rest between steps so we can see what's going on
        self.parent.clear() # makes sure only the most recent output is shown
        observation = self.parent.frog_body.look()  # now, when it looks,
                                                    # the frog will get a
                                                    # collection of several
                                                    # items

        # This creates a list of 'slot:value' pairs, using what's
        # contained in observation
        new_sight = []
        for slot in slots:
            val_index = observation[slot]   # gets a number indicating the state
                                            # of the object called 'slot'
            value = values[val_index]       # then looks it up
            new_sight.append(f'{slot}:{value}') # and adds it to

        # If there are any slots for door or key colour, add them to vision
        for slot in slots:
            col = f'{slot}_colour'
            if col in observation:
                new_sight.append(f'{col}:{observation[col]}')

        updated = ' '.join(new_sight)  # make the contents of new_memory into a
                                       # string ACT-R can understand
        # Show it in the printout
        print(f'the frog sees:\n  {updated}')
        print("the frog's goal:", objective)
        vision.set(updated)
        # this is the end of the look production

    def open_door(vision='key:held door:here'):
        print("the frog opens the door.")
        self.parent.frog_body.move("interact")
        goal.set('done')

    ######
    # END STATE

    # when all the flies are eaten, it's time to stop and rest
    def end(goal='done'):
        self.parent.clear()
        self.parent.frog_body.look()
        print("""
  ."|__________________  _,_
〈     To Be Continued ||\\|/
  '.|~~~~~~~~~~~~~~~~~~ ~~
  """)
        display(
            Audio(url='https://cdn.discordapp.com/attachments/965674740696092779/1072736721608921108/Studio_Project.mp3',
                   autoplay=True,
                   embed=True)
        )
        self.stop()

### Runs The Frog

In [ ]:
paradise_swamp = Pond() # Creates the environment for our frog

kermit = FrogMind()     # Creates the frog's brain
kermit.wait = time.sleep # A function for the frog to wait
kermit.Audio = Audio
kermit.display = display

# Places our frog as the agent in the swamp
paradise_swamp.agent = kermit  # put the frog's brain in the environment
paradise_swamp.start_world()   # start up the world
paradise_swamp.run()           # Annnnd action!

🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🪰🪰⬛🪰🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛🪰⬛⬛🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛🪰⬛⬛🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱🪰⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱🧱🧱🚪🧱🧱🧱⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛🧱    
🧱🧱🧱🧱🧱⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛🧱🧱🧱🧱🧱    
🧱⬛⬛🗝️🧱⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛🧱🗝️⬛⬛🧱    live frog reaction:
🧱⬛⬛⬛🧱⬛⬛⬛⬛⬛⬛🧱⬛🧱⬛⬛⬛⬛⬛🧱⬛⬛⬛🧱    ⬛🧱🟨🧱⬛⬛⬛
🧱⬛🧱🧱🧱⬛⬛⬛⬛⬛⬛🧱🟨🧱⬛⬛⬛⬛⬛🧱🧱🧱⬛🧱    ⬛🧱🟨🧱⬛⬛⬛
🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱🟨🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱    ⬛🧱🟨🧱⬛⬛⬛
🧱⬛🧱⬛⬛⬛⬛⬛⬛⬛⬛🧱🟨🧱⬛⬛⬛⬛⬛⬛⬛🧱⬛🧱    ⬛🧱🟨🧱⬛⬛⬛
🧱⬛🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🐸🧱🧱🧱🧱🧱🧱🧱🧱🧱⬛🧱    ⬛🧱🟨🧱⬛⬛⬛
🧱⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟨🟨🟨🟨🟨🟨🟨⬛⬛⬛⬛🧱    ⬛🧱🟨🧱🧱🧱🧱
🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱🧱    ⬛🧱🟨🐸🟨🟨🟨
eat all the flies
frog compass: ➡️
the frog sees:
  fly:nowhere key:nowhere door:nowhere wall:here
the frog's goal: key


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.gen_obs_grid to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.gen_obs_grid` for environment variables or `env.get_wrapper_attr('gen_obs_grid')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.agent_dir to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.agent_dir` for environment variables or `env.get_wrapper_attr('agent_dir')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.grid to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.grid` for environment variables or `env.get_wrapper_attr('grid'

## Your Task

Use your knowledge and work from Lab 3 to build up this frog. Your frog must complete four tasks, in order:

1.   Navigate the maze
2.   Check the colour of the door
3.   Find the key that matches the colour of the door
4.   Grab the key of the correct colour, bring it back to the door, and unlock the door.

Productions for `look`-ing around and `open_door` for unlocking the door have been provided. But you'll need to make production rules for navigating the maze, memorizing the colour of the door, finding the correct key and picking it up, as well as what to do if you find the *wrong* key.



---



### Task 1

#### Navigating the maze

Any maze can be solved using the **wall-following algorithm**. In any maze, pick either the left or right wall. Put your hand on that wall and never let your hand leave that wall. Keep walking forward, following the wall. Eventually, this will take you to the exit.

Picking the right-hand wall is called the "right-hand rule" and picking the left-hand hall is called the "left-hand rule". For our frog, we could use either, but the left-hand rule will result in our frog having an easier time solving this problem due to the frog's starting location: the frog will move towards the door before it goes to look at either key.

Below is a description of the wall following algorithm, stated in terms of what the frog needs to *do*, given what it *sees* in its immediate vicinity, using the `"object:location"` notation we're familiar with from the previous versions of the frog.


Location of Walls | How the Frog Moves | Explanation
--- | --- | ---
 `"wall:nowhere"` | turn `"left"` and then move `"forward"` | search for the left wall if you're not touching it
 `"wall:here"` | turn `"left"` and then move `"forward"` | search for the left wall if you're not touching it
 `"wall:right"` | turn `"left"` and then move `"forward"` | search for the left wall if you're not touching it
  `"wall:right_corner"` | turn `"left"` and then move `"forward"` | search for the left wall if you're not touching it
 `"wall:left"` | move `"forward"` | follow the left wall forward
 `"wall:hall"` | move `"forward"` | follow the left wall forward
 `"wall:left_corner"` | turn `"right"` | follow the left wall around turns

Note: `"wall:ahead"` is *not* used, don't worry about it.

You can read more about the wall-following algorithm at [Wikipedia](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Wall_follower) or this [blog post](https://andrewyong7338.medium.com/maze-escape-with-wall-following-algorithm-170c35b88e00).

##### **Create productions to make the frog perform the left-hand rule wall following algorithm.**

**Hint 1:** Use the movement production rules from the Lab 3 as templates for creating your production rules.

**Hint 2:** You can both turn left and move forward in one production rule by sending two commands to the frog's body in one production. Like so:

```python
        self.parent.frog_body.move('left')   # we turn left
        self.parent.frog_body.move('forward') # AND we hop forward
```

While this is easily achievable programmatically, ACT-R models typically only allow distinct actions to be represented by *distinct productions*, even if they occur in sequence. If you would like to try to do it that way, you may do so, although it is not required, since it complicates the exercise.

**Hint 3:** Don't forget all production rules should end with:

```python
        vision.set('unknown') # after each time the frog acts, it needs to look again
```

This ensures the frog will always update its `vision` buffer, which tells it about the current state of its world, and what action it should take.

**Hint 4:** Optionally, you can solve this with just 3 productions if you want to challege yourself. The trick is that you can write a production rule that matches to multiple wall states by using `"!"` which is the ACT-R symbol for "not", or logical negation. For example, if you wanted a production rule that fires whenever there is a wall to the frog's left, or a wall to both the frog's left and right (i.e., `"hall"`), we can specify that the production rule does **NOT** fire in every other situation:

```python
# only fires when wall:hall or wall:left
def rule_name(vision="wall:!nowhere!here!right!right_corner!left_corner"):
```

The new production, `rule_name`, will fire in the case that the wall is not `nowhere`, not `here`, not `right`, not `right_corner`, and not `left_corner`. So long as it is not any of those, `wall` may have any other value. Reviewing the possible values for the `wall` slot in the table above, we will note that the only values remaining are `left` and `hall`, and so it will fire in either case, and no others.

**Run the model to check to see if your production rules are working correctly.** (If the frog walks down the hall and gets stuck on the door then you have it working as intended.)


In [ ]:
#Task 1 - Navigate the maze
def turn_nowhere(vision = "wall:nowhere"):
  self.parent.frog_body.move('nowhere') #Starting point
  vision.set('unknown') #After each time the frog turns, we need it to look again

def turn_left(vision = "wall:here"): #Frog is going to stick to the left side
  self.parent.frog_body.move('left') #Frog needs to turn left
  vision.set('unknown')

def move_forward(vision = 'wall.left'): #Frog needs to move forward each time it turns
  self.parent.frog_body.move('left')
  vision.set('unknown')

def turn_right(vision = 'wall:right'):
  self.parent.frog_body.move('right')
  vision.set('unknown')

def turn_right_corner(vision = 'wall:right_corner'):
  self.parent.frog_body.move('right')
  vision.set('unknown')

def turn_left_corner(vision = 'wall:left_corner'):
  self.parent.frog_body.move('left')
  vision.set('unknown')

def turn_hall(vision = 'wall:hall'):
  self.parent.frog_body.move('right')
  vision.set('unknown')

def rule_name(vision = 'wall:nowhere!here!right!right_corner!left_corner'):
  if "left" in vision:
    self.parent.frog_body.move('right')
  else:
    self.parent.frog_body.move('forward')
  vision.set('unknown')




---

### Task 2

#### Check the colour of the door

Here we want to do two things:

1. We want the frog to memorize the colour of the door.
2. We want the frog to not get stuck on the door.

We can solve both of these problems with one production rule that fires as soon as the door is in sight. This production rule should add the colour of the door to long-term declarative memory and then turn the frog around 180˚ so it starts walking *away* from the door instead.

Let's define a production rule that fires whenever the door is in sight *and* creates a variable that holds the colour of the door.

The door can be `'left'`, `'right'`, `'ahead'`, `'here'`, or `'nowhere'`.

Thus, we can create a production rule that fires whenever the door is in sight by specifying `"door:!nowhere"`, i.e., the door is not nowhere. We can also create a variable that holds the colour of the door by using `"door_colour:?col"` where `"?col"` creates a variable called `col` inside our production's condition that stores the door's colour whenever the production fires:

```python
def remember_door(vision="door:!nowhere door_colour:?col"):
```

Now complete this production by doing the following:

- use `memory.add("door_colour:?col")` to add the colour of the door to long-term declarative `memory`
  - this is **only** possible *inside a production that fires on a condition containing* `?col`, as in the production you are currently creating, and will place the *value of* `col` in the `door_colour` slot
- turn the frog 180˚ so that it's pointing in the exact opposite direction
- add a print statement describing what this production rule makes the frog do
- set the vision buffer to `"unknown"`

**Hint:** a `"left"` or `"right"` turn is a 90˚ turn.

**Run the model to see if your new production rule is working.**

(If you have the Task 1 and Task 2 production rules working correctly, the frog should approach the door, see the colour of the door, turn around, and find one of the keys. At this point, the frog should stop in front of one of the keys and get stuck, because the frog doesn't know what to do with a key yet.)

In [ ]:
def remember_door(vision = "door:!nowhere door_colour:?col"):
  memory.add("door_colour:?col" + col)
  self.parent.frog_body.move('left')
  self.parent.frog_body.move('right')
  print("Frog sees the door, remembers the colour, and turns around")
  vision.set('unknown')
  remember_door.set('unknown')





---


### Task 3

When our template frog model is created, it is given an ACT-R `Memory()` object called `memory`, which is associated with a buffer called `recall`, which has an initial value of `"nothing"`. We have added chunks to memory (information about the door) using `memory.add()`, and now we need to retrieve that information so that the frog can check it against the keys. In order to retrieve it, we use `memory.request()`, which matches the contents of `memory` just like production conditions match to the contents of buffers, except, instead of firing a production, it places the retrieved chunk into its associated buffer, `recall`.

#### Find the key that matches the colour of the door

To pull this off, you need to use `memory.request()`, to check if the key is the right colour by comparing it to the memories of the door stored in long-term declarative memory. Here is how you can do that:

```python
memory.request('door_colour:?col')
```

This will retrieve a chunk in memory containing a slot called `door_colour`, whatever that slot's value, and place it in the `recall` buffer.

**You should design this production such that it fires *only* when the frog sees a key, *and* the** `recall` **buffer is currently empty.**

#### The frog must pick up the right key

We want to create a production that matches when the key is right in front of the frog and when the colour of the key agrees with the colour it remembers (as the key that matches the door colour is the key that unlocks the door). When the frog sees the right key, but is not directly in front of it, it should just continue following the wall until it reaches the key. There are three parts to this production: First, **we must test whether the colours agree and the key is in front of the frog**. Second, **the frog must pick up the key**. Third, **the frog must reset** `recall` **to its default value**, so as to avoid potential conflicts with other productions.

**Hint 1**

Once the frog recalls the colour of the door, it must compare that colour to the colour of the key it is looking at, and decide whether to pick up the key or leave it and go find the other one. More directly, the frog must check whether the value of `door_colour` in the `recall` buffer agrees with the value of `key_colour` in the `vision` buffer when the frog sees a key.

There is an easy way to create productions that test this sort of agreement in ACT-R. When you define a production, you may use a variable, such as `?col`, in multiple places. The production will then **only match in the case that all uses of that variable have an equal value**.

```python
def grab_key(vision="key:here key_colour:?col", recall='door_colour:?col'):
```

The above rule will only fire when the key is directly in front of the frog, since we don't want it to try to pick up the key, and when `key_colour` and `door_colour` have the same value.

**Hint 2**

Next, the frog should *pick up the key*. You've had the frog pick up keys in Lab 3, so if you don't remember how, go back to Lab 3 and check how it worked there.

**Hint 3**

Finally, the frog should *reset* `recall` to its default value. You have an example earlier in the template demonstrating how to set recall. It may be set just like any other buffer.

#### The frog must change directions if it is not the right colour

We also need to test if the frog is looking at the wrong key, and turn around immediately if that is the case. This requires a condition *similar* to the previous one, except: First, we do not want to wait until the key is directly in front of the frog, but rather make it turn 180˚ immediately, as soon as the key is recognized to be wrong. Second, the production should match when `key_colour` and `door_colour` are *not equal*.

**Hint 1**

There is also an easy way to accomplish this part in ACT-R. In this case, we write the condition exactly as before, but we add a `!` to note that the contents of one of the buffers is *not* whatever the variable `?col` captures.

```python
def wrong_key(vision="key_colour:?col", recall='door_colour:!?col'):
```

**Hint 2**

The above will match when `key_colour` has any value, so long as `door_colour` **does not have the same value**, marked by `!?col`. This production must cause the frog to *reset* `recall`, as before, and then *turn 180˚*.


In [ ]:
def remember_door_colour(vision = "key:!nowhere", recall = 'nothing'):
  memory.request('door_colour:?col')
  vision.set('unknown')

def grab_key(vision = "key:here key_colour:col?", recall = "door_colour:?col"):
  self.parent.frog_body.pick_up_key()
  recall.set('nothing')
  vision.set('unknown')

def wrong_key(vision = "key:here key_colour:?col", recall = "door_colour:!?col"):
  self.parent.frog_body.move('left')
  self.parent.frog_body.move('right')
  recall.set('nothing')
  vision.set('unknown')




---



### Task 4

Once the frog has grabbed the correct key, it will resume following the wall, and will eventually arrive back at the door. The final step is to make it open the door in the case that it is standing in front of the door and it has the correct key.

There is already a production in the template that fires when the frog is at the door and is holding a key, so you **don't need to add any new rules**. However, the productions you currently have fire **whether or not the frog has the key**, so the frog will not always open the door.

Your final task is to change the existing productions so that the frog always opens the door when it has the key. There are two ways you might go about this, and you can pick either one. (Or, experiment with both!)

1. As in Lab 3, you can make the frog ignore the door **only when the frog is not holding a key**. This is a simple change to the production where it memorizes the door's colour.
2. You can change the state of the `goal` buffer that was set up in the template when the frog picks up the key. This second approach is more orthodox for ACT-R. In this case, you should set the chunk in `goal`, as you would normally set a buffer, in `grab_key`, and then you should add a condition in the production where the frog memorizes the door, pertaining to what is in `goal`, such that the production can only fire when goal *does not* have the value you set in `grab_key`. You may also need to assign a default value to `goal` to make this work.

If you complete this task, the frog will print out an output clearly showing that you have completed the assignment. Test it a few more times to make sure your frog is working consistently, and if it does, **you may now submit this part of the assignment, and move on to the written portion**.

In [ ]:
def remember_door(vision = "door:!nowhere door_colour:?col", recall = 'nothing'):
  memory.add("door_colour:" + col)
  self.parent.frog_body.move('left')
  self.parent.frog_body.move('right')
  vision.set('nothing')
  vision.set('unknown')


# New Section